In [0]:
from __future__ import absolute_import, print_function
import numpy as np
import pyopencl as cl
import time


n = 9999999
a_np = np.random.rand(n).astype(np.float32)
b_np = np.random.rand(n).astype(np.float32)

ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

mf = cl.mem_flags

start_k = time.time()
a_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a_np)
b_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=b_np)

prg = cl.Program(ctx, """
__kernel void sum(
    __global const float *a_g, __global const float *b_g, __global float *res_g)
{
  int gid = get_global_id(0);
  res_g[gid] = a_g[gid] + b_g[gid];
}
""").build()

res_g = cl.Buffer(ctx, mf.WRITE_ONLY, a_np.nbytes)
prg.sum(queue, a_np.shape, None, a_g, b_g, res_g)  # a_np + b_np
res_np = np.empty_like(a_np)
cl.enqueue_copy(queue, res_np, res_g)

stop_k = time.time()
t1 = stop_k-start_k

print("Execution time for array size",n,"is", t1)